### _Importações necessárias_

In [67]:
import pandas as pd
from geopy.geocoders import Nominatim
from ipywidgets import fixed, interactive
import ipywidgets as wg
import plotly.express as px

pd.set_option('display.float_format', lambda x: '%.2f' % x)

### _Carregamento do arquivo a ser usado_

In [68]:
df = pd.read_csv('kc_house_data.csv')

# Mais algumas perguntas feitas pelo CEO

## 1. Qual a média do preço de compra dos imóveis por “Nível”?
- Nível 0 -> Preço entre 0 e 321.950
- Nível 1 -> Preço entre 321.950 e 450.000
- Nível 2 -> Preço entre 450.000 e 645.000
- Nível 3 -> Acima de 645.000

In [47]:
# Primeiramente é necessário a criação da coluna level e atribuir valores de 0 a 3.

# criação da coluna
df['level'] = 'NA'


In [48]:
# Atribuição dos valores
df['level_price'] = df['price'].apply(lambda n: 0 if n <= 321950 else
                                          1 if (n > 321950 ) & (n <= 450000) else
                                          2 if (n > 450000) & (n <= 650000 ) else
                                          3)  

In [49]:
# Média de preço por level

df[['price', 'level_price']].groupby('level_price').mean().reset_index()

,level_price,price
0,0,251557.65
1,1,385688.68
2,2,543019.37
3,3,998316.48


## 2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- **Size 0 -> Tamanho entre 0 e 1427 sqft**
- **Size 1 -> Tamanho entre 1427 e 1910 sqft**
- **Size 2 -> Tamanho entre 1910 e 2550 sqft**
- **Size 3 -> Tamanho acima de 2550 sqft**


In [50]:
# Primeiramente é necessário a criação da coluna size e atribuir valores de 0 a 3.

# criação da coluna
df['level'] = 'NA'

# Atribuiçao dos valores
df['size'] = df['sqft_living'].apply(lambda n: 0 if n <= 1427 else
                                          1 if (n > 1427 ) & (n <= 1910) else
                                          2 if (n > 1910) & (n <= 2550 ) else
                                          3) 

In [51]:
# Média de sqft living por size


df[['sqft_living', 'size']].groupby('size').mean().reset_index()

,size,sqft_living
0,0,1123.83
1,1,1664.96
2,2,2211.79
3,3,3329.61


## 3. Adicione as seguinte informações ao conjunto de dados original:
- **Place ID: Identificação da localização**
- **OSM Type: Open Street Map type**
- **Country: Nome do País**
- **Country Code: Código do País**


# _Primeiro método usando o condicional while, método esse mais demorado e contraproducente._


In [52]:
# Inicializando a Nominatim API
geo_locator = Nominatim(user_agent='geo_content_get')

# Criação das novas colunas
df[['place_id', 'osm_type', 'country', 'country_code']] = 'NA'


In [53]:
len(df)

21613

### Por haver demora da coleta de dados na API estava dando Time Out e tive que fazer um tratamento de erro
### Achei melhor optar pelo while para que ele se mantivesse no laço até que a coleta do dado fosse realizada
```py
count = 0
while count < len(df):
    try:
        response = geo_locator.reverse(f"{df.loc[count, 'lat']},{df.loc[count, 'long']}")
        df.loc[count, 'place_id'] = response.raw['place_id']
        df.loc[count, 'osm_type'] = response.raw['osm_type']
        df.loc[count, 'country'] = response.raw['address']['country']
        df.loc[count, 'country_code'] = response.raw['address']['country_code']
        print(response.raw['place_id'], response.raw['osm_type'], response.raw['address']['country'], response.raw['address']['country_code'])
    except:
        continue
    count+=1
while_progress    
print('fim')

df[['place_id', 'osm_type', 'country', 'country_code']]
```

# _Segundo método usando Multithreds_ 

### Bibliotecas necessárias

In [133]:
from multiprocessing import Pool # necessária para criação dos workers

### É necessári a criação de uma função para fazer o trabalho de multithreding

#### Função que esta dentro do módulo df_constructor

```py
from time import sleep
from geopy.geocoders import Nominatim

# Criação do geo_locator
geo_locator = Nominatim(user_agent="geoTantoFaz")

def get_data(new_df): 
    index, row = new_df
    sleep(1)
    # Chamada da API
    response = geo_locator.reverse(new_df['location'])
    
    address = response.raw['address']
    place_id = response.raw['place_id']
    osm_type = response.raw['osm_type']
    country = address['country']
    country_code = address['country_code']
    return place_id, osm_type, country, country_code
```

#### Importante frizar que esta função vai ser criada fora do JN, a função acima é só uma ilustração da função que esta dentro de um módulo chamado df_constructor. Foi criada fora devido aos problemas que o JN tem com Multithreding.

## Agora vamos a criação do dataframe com as informações solicitadas

In [134]:
# Criada a coluna location com as coordenadas 

df['location'] = df[['lat', 'long']].apply(lambda zeta: f"{zeta['lat']},{zeta['long']}", axis=1)

In [135]:
# Criação dos workers

workers = Pool(2) # Haverá 2 cores trabalhando

### Módulo criado anteriormente

In [136]:
import df_constructor

In [137]:
# Criando o dataframe com os dados

new_df = df[['id', 'location']].head()


new_df[['place_id', 'osm_type', 'country', 'country_code']] = workers.map(df_constructor.get_data, new_df.iterrows())


TypeError: tuple indices must be integers or slices, not str

## 4. Adicione os seguinte filtros no Mapa:
- **Tamanho mínimo da área da sala de estar.**
- **Número mínimo de banheiros.**
- **Valor Máximo do Preço.**
- **Tamanho máximo da área do porão.**
- **Filtro das Condições do Imóvel.**
- **Filtro por Ano de Construção.**


In [54]:
style = {'description_width': 'initial'}

In [55]:
df['bathrooms'].unique()

array([1.  , 2.25, 3.  , 2.  , 4.5 , 1.5 , 2.5 , 1.75, 2.75, 3.25, 4.  ,
       3.5 , 0.75, 4.75, 5.  , 4.25, 3.75, 0.  , 1.25, 5.25, 6.  , 0.5 ,
       5.5 , 6.75, 5.75, 8.  , 7.5 , 7.75, 6.25, 6.5 ])

In [56]:
# Botões iterativos 
# Para mais botões visitar o site do ipywidgets

# Tamanho da sala de estar

living_room_min = widgets.IntSlider(
    value = 2079,
    min = 290,
    max = 13540,
    step = 1,
    description = 'Tamanho da sala de estar',
    disable = False,
    style = style)

# Número máximo de banheiros

bathrooms_numbers = widgets.Dropdown(
    options= [1.  , 2.25, 3.  , 2.  , 4.5 , 1.5 , 2.5 , 1.75, 2.75, 3.25, 4.  ,
       3.5 , 0.75, 4.75, 5.  , 4.25, 3.75, 0.  , 1.25, 5.25, 6.  , 0.5 ,
       5.5 , 6.75, 5.75, 8.  , 7.5 , 7.75, 6.25, 6.5 ],
    value= 2,
    description='Número máximo de banheiros:',
    disabled=False,
    style = style
)

# Preço Máximo

max_price = widgets.IntSlider(
    value = 540088,
    min = 75000,
    max = 7700000,
    step = 1,
    description = 'Preço máximo',
    disable = False,
    style = style)

# Tamanho máximo do porão

max_sqft_basement = widgets.IntSlider(
    value = 291,
    min = 0,
    max = 4820,
    step = 1,
    description = 'Tamanho máximo do porão',
    disable = False,
    style = style)

# Condição do imóvel

house_condition = widgets.Dropdown(
    options= [3, 5, 4, 1, 2],
    value= 2,
    description='Condição do imóvel:',
    disabled=False,
    style = style
)

# Ano de construção

btn_yr_built = widgets.IntSlider(
    value = 1971,
    min = 1900,
    max = 2015,
    step = 1,
    description = 'Ano de construção',
    disable = False,
    style = style
    )


In [57]:
# Criação do Mapa

def update_map(df, lrm, bn, mp, msb, hc, byb):
    
    filtro = df[(df['sqft_living'] <= lrm) 
               & (df['bathrooms'] == bn)
               & (df['price'] == mp)
               & (df['sqft_basement'] == msb)
               & (df['condition'] == hc)
               & (df['yr_built'] == byb)
    ][['id', 'lat', 'long', 'level_price', 'size', 'price', 
       'bathrooms', 'sqft_basement', 'yr_built', 'condition', 'sqft_living']]
    mapa = px.scatter_mapbox(filtro, 
                      lat='lat', 
                      lon='long',
                      hover_name='id',
                      size='level_price',
                      color='size',
                      hover_data=['price'],
                      color_discrete_sequence=['grey'],
                      size_max=15,
                      zoom=10,
                     )
    mapa.update_layout(mapbox_style='open-street-map')
    mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
    mapa.show();

In [58]:
wg.interactive(update_map, df=wg.fixed(df), lrm=living_room_min, bn=bathrooms_numbers, mp=max_price, msb=max_sqft_basement, hc=house_condition, byb=btn_yr_built)

interactive(children=(IntSlider(value=2079, description='Tamanho da sala de estar', max=13540, min=290, style=…

## 5. Adicione os seguinte filtros no Dashboard:
- **Filtro por data disponível para compra.**
- **Filtro por ano de renovação.**
- **Filtro se possui vista para a água ou não.**